# SpotifyDB Notebook
This notebook demonstrates setting up and using the SpotifyDB utilities.

In [1]:
#in powershell
%pip install spotipy duckdb python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## create cred.py  in the root directory 
add your own client id and secrets using the values you have optained from spotify.


### cred.py
```python
# get values from 
# https://developer.spotify.com/dashboard/
spotify_client_id = "your id"
spotify_client_secret = "your secret"
```


In [2]:
# import the credentials from cred.py
from cred import spotify_client_id, spotify_client_secret

## import libraries

In [3]:
import os
from dotenv import load_dotenv

import duckdb
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# custom functions
import functions as fn


## setting up the environment, api, and duckdb

In [4]:
# Load .env
load_dotenv()

CLIENT_ID = spotify_client_id
CLIENT_SECRET = spotify_client_secret
REDIRECT_URI = "http://127.0.0.1:8000/callback"
SCOPE = (
    "user-library-read user-follow-read playlist-read-private playlist-modify-private playlist-modify-public"
)

assert CLIENT_ID and CLIENT_SECRET and REDIRECT_URI, "Missing Spotify env vars"

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE,
        open_browser=True,  # will open auth page in browser
        cache_path=".cache-spotifydb"  # token cache
    )
)

current_user = sp.current_user()
current_user["display_name"], current_user["id"]


('1257524228', '1257524228')

In [5]:
# In-memory DB (great for playing around)
# con = duckdb.connect(database=':memory:')

# If you want persistent on-disk:
con = duckdb.connect(database='spotify.duckdb')


In [13]:
df_tables = con.execute(f"SHOW TABLES").df()
display(df_tables)


,name
0,_temp_df
1,followed_artist
2,my_liked_songs
3,table_updated


## geting basic data 

### my followed artist 

In [9]:


table_age = fn.duckdb_table_age(con, "followed_artist")

print(f"table age: {table_age} days old")

if table_age is None or table_age > 1.0:
    print("Refreshing followed_artist table...")

    followed_artist = fn.get_followed_artists_df(sp)
    display(followed_artist.head())

    print("Loading followed_artist table from DuckDB...")
    fn.df_to_duckdb(con, followed_artist, "followed_artist")
else:
    followed_artist = fn.duckdb_to_df(con, "followed_artist")
    print("Loaded followed_artist table from DuckDB.")


display(f"Followed Artists: {len(followed_artist)} records")
display(followed_artist.head())


table age: 0.002500417293221862 days old
Loaded followed_artist table from DuckDB.


'Followed Artists: 47 records'

,artist_id,name,followers,popularity,genres,uri
0,7I95CM75shzCjHuTzrepjM,Nova Twins,181046,51,,spotify:artist:7I95CM75shzCjHuTzrepjM
1,2qybVy8UqYXDx6x6BZedPE,Shadow Realm,229,10,,spotify:artist:2qybVy8UqYXDx6x6BZedPE
2,2mKV7sPvlBvzyPLeZhzT6q,Skyebrows,540,25,,spotify:artist:2mKV7sPvlBvzyPLeZhzT6q
3,2gTWLwanLiNZFR0iNPD847,EZ Band,119593,43,"latin country, tejano, norteño, cumbia norteña",spotify:artist:2gTWLwanLiNZFR0iNPD847
4,5qJEtz7aC2nwA9LsjfkGVM,Haku.,168869,44,japanese indie,spotify:artist:5qJEtz7aC2nwA9LsjfkGVM


### my liked songs

In [8]:

try:
    my_liked_songs = fn.load_my_saved_tracks(sp)
    display(f"My Liked Songs: {len(my_liked_songs)}")
    display(my_liked_songs.head())

    fn.df_to_duckdb(con, my_liked_songs, "my_liked_songs")
except Exception as e:
    print("Error loading My Liked Songs:", e)
    my_liked_songs = fn.duckdb_to_df(con, "my_liked_songs")


'My Liked Songs: 1734'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,13,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
1,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
2,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI
3,2025-11-20T00:04:10Z,3ZrZYpxHQiBgDpfxHkTnLC,Elon's Musk,Elon's Musk,3BZNhRBQknND6A7jjfE8lT,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,220536,False,42,False,spotify:track:3ZrZYpxHQiBgDpfxHkTnLC
4,2025-10-12T21:43:37Z,1gR5TDDEvDkxTs1UGjR910,what i got - lofi,study and chill with sublime,696pAMUhtOEyenQukQKRxI,"lonelyboy, Sublime",30pF2pv50VXX6TgiMmKKjc,133333,False,21,False,spotify:track:1gR5TDDEvDkxTs1UGjR910


In [10]:
table_age = fn.duckdb_table_age(con, "my_liked_songs")

print(f"table age: {table_age} days old")

if table_age is None or table_age > 1.0:
    print("Refreshing my_liked_songs table...")

    my_liked_songs = fn.load_my_saved_tracks(sp)
    display(my_liked_songs.head())

    print("Loading my_liked_songs table from DuckDB...")
    fn.df_to_duckdb(con, my_liked_songs, "my_liked_songs")
else:
    my_liked_songs = fn.duckdb_to_df(con, "my_liked_songs")
    print("Loaded my_liked_songs table from DuckDB.")


display(f"my_liked_songs: {len(my_liked_songs)} records")
display(my_liked_songs.head())

table age: None days old
Refreshing my_liked_songs table...


,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,18,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
1,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
2,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI
3,2025-11-20T00:04:10Z,3ZrZYpxHQiBgDpfxHkTnLC,Elon's Musk,Elon's Musk,3BZNhRBQknND6A7jjfE8lT,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,220536,False,42,False,spotify:track:3ZrZYpxHQiBgDpfxHkTnLC
4,2025-10-12T21:43:37Z,1gR5TDDEvDkxTs1UGjR910,what i got - lofi,study and chill with sublime,696pAMUhtOEyenQukQKRxI,"lonelyboy, Sublime",30pF2pv50VXX6TgiMmKKjc,133333,False,21,False,spotify:track:1gR5TDDEvDkxTs1UGjR910


Loading my_liked_songs table from DuckDB...


'my_liked_songs: 1734 records'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,18,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
1,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
2,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI
3,2025-11-20T00:04:10Z,3ZrZYpxHQiBgDpfxHkTnLC,Elon's Musk,Elon's Musk,3BZNhRBQknND6A7jjfE8lT,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,220536,False,42,False,spotify:track:3ZrZYpxHQiBgDpfxHkTnLC
4,2025-10-12T21:43:37Z,1gR5TDDEvDkxTs1UGjR910,what i got - lofi,study and chill with sublime,696pAMUhtOEyenQukQKRxI,"lonelyboy, Sublime",30pF2pv50VXX6TgiMmKKjc,133333,False,21,False,spotify:track:1gR5TDDEvDkxTs1UGjR910


## new_liked_songs

In [11]:
new_liked_songs = duckdb.query(
"""
select * from my_liked_songs 
order by saved_at 
desc limit 100
"""
).to_df()

display(f"new_liked_songs: {len(new_liked_songs)}")
display(new_liked_songs.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**New Liked Songs",
    description="My 100 most recently liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, new_liked_songs["uri"].tolist())


'new_liked_songs: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,18,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
1,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
2,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI
3,2025-11-20T00:04:10Z,3ZrZYpxHQiBgDpfxHkTnLC,Elon's Musk,Elon's Musk,3BZNhRBQknND6A7jjfE8lT,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,220536,False,42,False,spotify:track:3ZrZYpxHQiBgDpfxHkTnLC
4,2025-10-12T21:43:37Z,1gR5TDDEvDkxTs1UGjR910,what i got - lofi,study and chill with sublime,696pAMUhtOEyenQukQKRxI,"lonelyboy, Sublime",30pF2pv50VXX6TgiMmKKjc,133333,False,21,False,spotify:track:1gR5TDDEvDkxTs1UGjR910


## cream of crop

In [12]:
cream_of_crop = duckdb.query(
"""
select * from my_liked_songs 
order by popularity 
desc limit 100
"""
).to_df()

display(f"cream_of_crop: {len(cream_of_crop)}")
display(cream_of_crop.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**Cream of Crop",
    description="My 100 most popular liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, cream_of_crop["uri"].tolist())

'cream_of_crop: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-09-03T18:14:36Z,1CPZ5BxNNd0n0nF4Orb9JS,Golden,KPop Demon Hunters (Soundtrack from the Netfli...,14JkAa6IiFaOh5s0nMyMU9,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",2yNNYQBChuox9A5Ka93BIn,194607,False,98,False,spotify:track:1CPZ5BxNNd0n0nF4Orb9JS
1,2015-07-20T21:23:34Z,57bgtoPSgt236HzfBOd8kj,Thunderstruck,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,AC/DC,711MCceyCBcFnzjGY4Q7Un,292333,False,87,False,spotify:track:57bgtoPSgt236HzfBOd8kj
2,2016-09-30T23:35:05Z,3ZOEytgrvLwQaqXreDs2Jx,Can't Stop,By the Way (Deluxe Edition),6deiaArbeoqp1xPEGdEKp1,Red Hot Chili Peppers,0L8ExT028jH3ddEcZwqJJ5,269000,False,87,False,spotify:track:3ZOEytgrvLwQaqXreDs2Jx
3,2019-09-21T18:43:08Z,6ECp64rv50XVz93WvxXMGF,This Love,Songs About Jane: 10th Anniversary Edition,5zClcGCSWj926AMjvBNSLc,Maroon 5,04gDigrS5kc9YWfZHwBETP,206200,False,87,False,spotify:track:6ECp64rv50XVz93WvxXMGF
4,2019-01-12T18:41:51Z,7BKLCZ1jbUBVqRi2FVlTVw,Closer,Closer,0rSLgV8p5FzfnqlEk4GzxE,"The Chainsmokers, Halsey",69GGBxA162lTqCwzJG5jLp,244960,False,87,False,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw


## discover_these 

In [ ]:
top_artist = followed_artist.head(10)['artist_id'].to_list()

disc_these = pd.DataFrame()
for artist_id in top_artist:
    artist_albums = fn.get_artist_top_tracks_df(sp, artist_id)
    disc_these = pd.concat([disc_these, artist_albums], ignore_index=True)

display(f"disc_these: {len(disc_these)}")
display(disc_these.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**discover these",
    description="Best songs from recently followed artist, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, new_liked_songs["uri"].tolist())


'disc_these: 82'

,track_id,name,album_name,album_id,artist_name,artist_ids,popularity,duration_ms,explicit,preview_url,uri
0,6sIbv1oWOuma2qV907MUbk,1x1 (feat. Nova Twins),POST HUMAN: SURVIVAL HORROR,0e1WaSNDZnoPixaxDNdWo4,"Bring Me The Horizon, Nova Twins","1Ffb6ejR6Fe5IamqA5oRUF, 7I95CM75shzCjHuTzrepjM",66,209598,True,None,spotify:track:6sIbv1oWOuma2qV907MUbk
1,4WWogojK9PF7sbMvmqXGMV,Glory,Parasites & Butterflies,5uRq9tAqhsULMA35OL37Uk,Nova Twins,7I95CM75shzCjHuTzrepjM,46,222373,False,None,spotify:track:4WWogojK9PF7sbMvmqXGMV
2,3XRknZ0U5NKW8r5g44ptA6,Antagonist,Supernova,41ycYGcnhkDb3pFkL8vSPJ,Nova Twins,7I95CM75shzCjHuTzrepjM,44,186173,True,None,spotify:track:3XRknZ0U5NKW8r5g44ptA6
3,2tifAgNsyjjL20h1QKtVTe,Monsters,Parasites & Butterflies,5uRq9tAqhsULMA35OL37Uk,Nova Twins,7I95CM75shzCjHuTzrepjM,39,211786,False,None,spotify:track:2tifAgNsyjjL20h1QKtVTe
4,34yGnRiItM9kXHCXdYoQ2p,N.O.V.A,Parasites & Butterflies,5uRq9tAqhsULMA35OL37Uk,Nova Twins,7I95CM75shzCjHuTzrepjM,40,160960,True,None,spotify:track:34yGnRiItM9kXHCXdYoQ2p
